# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
import re
import time

import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from xgboost import XGBClassifier

from sqlalchemy import create_engine

/Users/kiril/opt/anaconda3/envs/Project-2/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [4]:
nltk.download(['punkt',
               'wordnet',
               'averaged_perceptron_tagger',
               'omw-1.4',
               'stopwords'])

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

np.random.seed(42)

engine = create_engine('sqlite:///../Data/DisasterResponse.db')

[nltk_data] Downloading package punkt to /Users/kiril/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/kiril/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kiril/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /Users/kiril/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/kiril/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# load data from database
df = pd.read_sql_table('DisasterResponse', engine)
print( f'Original shape                        {df.shape}')

non_categories = ['id', 'message', 'original', 'genre']

categories = df.columns.drop(non_categories)
    
# Split in features and targets
X = df['message'].values

Y = df.drop(non_categories, axis=1).values

categories = df.columns.drop(non_categories)

print( f'\nShape of X {X.shape}')
print( f'Shape of Y {Y.shape}')

Original shape                        (26216, 40)

Shape of X (26216,)
Shape of Y (26216, 36)


### 2. Write a tokenization function to process your text data

In [6]:
url_regex = r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*'
url_regex_comp = re.compile(url_regex, re.IGNORECASE)

letters_regex = r'[^a-zA-Z0-9]'
letters_regex_comp = re.compile(letters_regex, re.IGNORECASE)
    
def tokenize(text):
   
    text = url_regex_comp.sub("urlplaceholder", text)

    text = letters_regex_comp.sub(" ", text)

    # tokenize text
    tokens = word_tokenize(text)
   
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
stop_words = set(tokenize(' '.join(stopwords.words('english'))))

count_vectorizer = CountVectorizer(tokenizer=tokenize,
                                   stop_words=stop_words, max_df=0.5
                                  )

tf_idf = TfidfTransformer(smooth_idf=True)

tf_idf_vect = Pipeline([('vect', count_vectorizer),
                        ('tf_idf', tf_idf)
                       ])


classifier = LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1)

pipeline = Pipeline([('prep', tf_idf_vect), 
                     ('clf', MultiOutputClassifier(classifier))
                    ],
                    verbose=1
                   )

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

start_time = time.time()
pipeline.fit(X_train, Y_train)
print("--- %s seconds ---" % (time.time() - start_time))


[Pipeline] .............. (step 1 of 2) Processing prep, total=   6.8s
[Pipeline] ............... (step 2 of 2) Processing clf, total=  28.0s
--- 34.86955213546753 seconds ---


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
def eval_model(model, X_test, categories):
    Y_pred = model.predict(X_test)

    print(classification_report(Y_test, Y_pred, target_names=categories))

In [10]:
eval_model(pipeline, X_test, categories)

                        precision    recall  f1-score   support

               request       0.60      0.74      0.67      1111
                 offer       0.05      0.09      0.06        33
           aid_related       0.72      0.74      0.73      2670
          medical_help       0.37      0.65      0.48       535
      medical_products       0.34      0.64      0.44       344
     search_and_rescue       0.16      0.40      0.23       159
              security       0.08      0.17      0.11       116
              military       0.38      0.67      0.48       200
                 water       0.58      0.86      0.69       418
                  food       0.73      0.85      0.78       745
               shelter       0.56      0.79      0.65       581
              clothing       0.40      0.67      0.50        98
                 money       0.27      0.66      0.39       133
        missing_people       0.18      0.42      0.26        73
              refugees       0.27      

/Users/kiril/opt/anaconda3/envs/Project-2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kiril/opt/anaconda3/envs/Project-2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# specify parameters for grid search

params = [
    {   
        'prep__vect__max_df': [0.5, 1.0],
        'prep__vect__max_features': [300, 1000, 1500],
        'prep__vect__min_df': [1, 5, 10],
                
        'clf__estimator': [LogisticRegression()],   
        'clf__estimator__class_weight': ['balanced'],
        'clf__estimator__C': [0.1, 1, 10],
        'clf__estimator__n_jobs': [-1],
        'clf__estimator__max_iter': [1000],
    },

    
    {   
        'prep__vect__max_df': [0.5, 1.0],
        'prep__vect__max_features': [300, 1000, 1500],
        'prep__vect__min_df': [1, 5, 10],
                
        'clf__estimator': [XGBClassifier()],   
        'clf__estimator__scale_pos_weight': [10, 25, 50],
        'clf__estimator__n_jobs': [-1],
        'clf__estimator__objective': ['binary:logistic'],
        'clf__estimator__eval_metric': ['auc'],
    },
]


# create grid search object
cv1 = GridSearchCV(pipeline, 
                   param_grid=params,
                   cv=3,
                   scoring='f1_macro',
                   verbose=1,
                   n_jobs=-1
                  )

In [ ]:
start_time = time.time()
cv1.fit(X_train, Y_train)
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
print("\nBest Parameters:", cv1.best_params_)

In [ ]:
print("\nBest Score:", cv1.best_score_)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
eval_model(cv1, X_test, categories)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [ ]:
class WordVectorVectorizer(TransformerMixin,BaseEstimator):
    def __init__(self, model="en_core_web_lg"):
        self.model = model

    def fit(self,X,y=None):
        return self

    def transform(self,X):
        nlp = spacy.load(self.model)
        return np.concatenate([nlp(doc).vector.reshape(1,-1) for doc in X])
    
    
class MessageLengthTransformer(TransformerMixin,BaseEstimator):
#     def __init__(self):
#         pass
        
    def fit(self,X,y=None):
        return self

    def transform(self,X):
        return np.reshape([len(doc) for doc in X], (-1, 1))
    
    
class NumberOfUrlsTransformer(TransformerMixin,BaseEstimator):
    def __init__(self):
        url_regex = '((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*'
        self.url_regex_comp = re.compile(url_regex, re.IGNORECASE)
        
    def fit(self,X,y=None):
        return self

    def transform(self,X):
        return np.reshape([len(re.findall(self.url_regex_comp, doc)) for doc in X], (-1, 1))

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion

stop_words = set(tokenize(' '.join(stopwords.words('english'))))

scaler = StandardScaler()

count_vectorizer = CountVectorizer(tokenizer=tokenize,
                                   stop_words=stop_words,
#                                    max_df=0.5,
#                                    min_df=5,
#                                    max_features=500
                                  )

tf_idf = TfidfTransformer()

tf_idf_vect = Pipeline([('vect', count_vectorizer),
                        ('tf_idf', tf_idf)
                       ])

message_length = Pipeline([('message_length', MessageLengthTransformer()),
                           ('scaler', scaler)
                          ])

number_of_urls = Pipeline([('number_of_urls', NumberOfUrlsTransformer()),
                           ('scaler', scaler)
                          ])

# word_vect = WordVectorVectorizer()



feature_union = FeatureUnion([
#     ('word_vect', word_vect),
    ('tf_idf_vect', tf_idf_vect),
    ('message_length', message_length),
    ('number_of_urls', number_of_urls),
])

classifier = LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1)

pipeline = Pipeline([('feature_union', feature_union),
                     ('scaler', StandardScaler(with_mean=False)),
                     ('clf', MultiOutputClassifier(classifier))],
                    verbose=1)

In [ ]:
# specify parameters for grid search

params = [
    {   
        'clf__estimator': [LogisticRegression()],   
        'clf__estimator__class_weight': ['balanced'],
        'clf__estimator__n_jobs': [-1],
        'clf__estimator__penalty': ['l2'],
        'clf__estimator__max_iter': [1000],

    },

]


# create grid search object
cv = GridSearchCV(pipeline,
                  param_grid=params,
                  cv=2,
                  scoring='f1_macro',
                  verbose=1,
                  n_jobs=-1
                 )

In [ ]:
start_time = time.time()
cv.fit(X_train, Y_train)
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
eval_model(cv, X_test, categories)

### 9. Export your model as a pickle file

In [ ]:
import pickle

with open('classifier.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.